In [3]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.layers.experimental.preprocessing as preprocessing

# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import pandas as pd
import glob
from PIL import Image

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory

plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')

In [ ]:
# Kaggle competition restrictions
#!pip install ../input/efficientnet/Keras_Applications-1.0.8-py3-none-any.whl
#!pip install ../input/efficientnet/efficientnet-1.1.1-py3-none-any.whl

!pip install Keras-Applications
!pip install efficientnet


from efficientnet import tfkeras as efn

In [ ]:
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
IMAGE_CHANNELS = 3
BATCH_SIZE = 28
EPOCHS = 100

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

data = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv",dtype=str) 
sample = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv",dtype=str)

data['label'] = data['label'].astype(str)

train_datagen = ImageDataGenerator(validation_split=0.25, zoom_range=0.2, rotation_range=0.25, vertical_flip=True, horizontal_flip=True)
valid_datagen = ImageDataGenerator(validation_split=0.25)

In [ ]:
data

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
    dataframe = data,
    directory="../input/cassava-leaf-disease-classification/train_images/",
    x_col="image_id",
    y_col="label",
    subset="training",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="sparse",
    target_size=(IMAGE_WIDTH,IMAGE_HEIGHT)
)

valid_generator=valid_datagen.flow_from_dataframe(
    dataframe = data,
    directory="../input/cassava-leaf-disease-classification/train_images/",
    x_col="image_id",
    y_col="label",
    subset="validation",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="sparse",
    target_size=(IMAGE_WIDTH,IMAGE_HEIGHT)
)

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
check_pointer = ModelCheckpoint(
    filepath="./chckpt.h5", 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True,
    save_weights_only = False
)

early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.001, 
    patience=10, 
    verbose=1,
    restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    patience=5, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.0001
)

callbacks = [check_pointer, early_stopping, learning_rate_reduction]

In [ ]:
# Replace with this line for offline weights loading
# ../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b3_noisy-student_notop.h5
efficientNet = efn.EfficientNetB3(include_top=False, weights='noisy-student', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
efficientNet.trainable = True

In [ ]:
from tensorflow.keras.models import Model, Sequential

model = keras.Sequential([
    efficientNet,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax'),
])

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tfa.optimizers.Lookahead(tfa.optimizers.RectifiedAdam()),
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [check_pointer, early_stopping, learning_rate_reduction]
)

In [ ]:
model.save('./model.h5')

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()
history_frame.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
ss

In [ ]:
preds = []

for image_id in ss.image_id:
    image = Image.open(os.path.join('../input/cassava-leaf-disease-classification/',  "test_images", image_id))
    image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))

ss['label'] = preds
ss

In [ ]:
ss.to_csv('submission.csv', index = False)